# Create GUI to predict digits -(Need to amend some situation)

### Import libraries

In [1]:
import os
import PIL
import cv2
import glob
import numpy as np
from tkinter import *
from PIL import Image, ImageDraw, ImageGrab
import tensorflow as tf
from tensorflow import keras

### Load Model

In [2]:
model = keras.models.load_model(r'C:\Users\bZm Graphics\Beginner Coding in Py3\ML\MNIST\MNIST_test Purpose/model.h1')
print('Model load successfully, Go for the app')

Model load successfully, Go for the app


### Source
https://medium.com/analytics-vidhya/handwritten-digit-recognition-gui-app-46e3d7b37287

In [3]:
### Creating Mainloop()
# Create a main window first (named as root).
root = Tk()
root.resizable(0,0)
root.title("Handwritten Digit Recognition GUI App")


#Initialize few variables
lastx, lasty = None, None
image_number = 0

#create a canvas for drawing
cv = Canvas(root, width=640, height =480, bg='white')
cv.grid(row=0, column=0, pady=2, sticky=W, columnspan=2)


# #Tkinter provides a powerful mechanism to let you deal with events yourself.
cv.bind('<Button-1>', activate_event)

#Add Buttons and Labels
btn_save = Button(text="Recognize Digit", command=Recognize_Digit)
btn_save.grid(row=2, column=0, pady=1, padx=1)
button_clear = Button(text = "Clear Widget", command = clear_widget)
button_clear.grid(row=2, column=1, pady=1, padx=1)

# mainloop() is used when your application is ready to run.
root.mainloop()



### creating function
def clear_widget():
    global cv
    # To clear a canvas
    cv.delete("all")
    
def activate_event(event):
    global lastx, lasty
    # <B1-Motion>
    cv.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event_y
    
def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event_y
    #do the canvas drawings
    cv.create((lastx, lasty, x, y),width=8, fill='black',capstyle=Round, smooth=True, splinesteps=12)
    lastx, lasty = x, y
    

    
    
### Creating Recognition_Digit()
def Recognize_Digit():
    global image_number
    predictions = []
    percentage = []
    #image_number = 0
    filename = f'image_{image_number}.png'
    widget=cv
    
    #get the widget co-ordinates
    x=root.winfo_rootx() + widget.winfo_x()
    y=root.winfo_rooty() + widget.winfo_y()
    x1=x+widget.winfo_width()
    y1=y+widget.winfo_width()
    
    
    #grab the image, crop it according to my requirement and save it in png format
    ImageGrab.grab().crop((x,y,x1,y1)).save(filename)
    
    
    
    
    
    ### OpenCV to find contours of an image
    #It is better to use binary images fopr better accuracy so before finding contours, apply a threshold.

    # read the image in color format
    image = cv2.imread(filename, cv2.IMREAD_COLOR)
    # Convert the image in grayscale
    gray = cv2.cvtcolor(image, cv2.COLOR_BGR2GRAY)
    # applying otsu thresholding
    ret,th = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    # findContour() function helps in extraxting the contours from the image.
    contours = cv2.findContours(th, cv.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    

    
    
    
### Creating bounding boxes for contours and extract ROI. After extracting region of interest, we will preprocess(resize,reshape and normalize) the image to support our model input. Then execute model.predict()
for cnt in contours:
    # Get bounding boxes and extract ROI
    x,y,w,h = cv2.boundingRect(cnt)
    # Create rectangle
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 1)
    top = int(0.05 * th.shape[0])
    bottom = top
    left = int(0.05 * th.shape[1])
    right = left
    th_up = cv2.copyMakeBorder(th, top, bottom, left, right, cv2.BORDER_REPLICATE)
    # Extract the image ROI
    roi= th[y-top:y+h+bottom, x-left:x+w+right]
    # resize roi image to 28*28 pixels
    img = cv2.resize(roi, (28,28), interpolation=cv2.INTER_AREA)
    # reshaping the image to support our model input
    img = img.reshape(1,28,28,1)
    # normalizing the image to support our model input
    img = img/255.0
    # its time to predict the result
    pred = model.predict([img])[0]
    # numpy.argmax(input array) returns the indicates of the maximum values.
    final_pred = np.argmax(pred)
    data = str(final_pred) +' '+ str(int(max(pred)*100))+'%'
    # cv2.putText() method is used to draw a text string on image.
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    color = (255, 0, 0)
    thickness = 1
    cv2.putText(image, data, (x,y-5), font, fontScale, color, thickness)

        
# Showing the predicted results on new window.
cv2.imshow('image', image)
cv2.waitkey(0)

NameError: name 'activate_event' is not defined